In [52]:
from sympy import Symbol, Matrix
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider

# Define symbols
M12 = Symbol('(rE)')
M11 = Symbol('(1/no^2)')
M22 = Symbol('(1/ne^2)')

# Define the matrix
M = Matrix([[M11, M12], [M12, M22]])

# Calculate eigenvalues and eigenvectors
eigen_data = M.eigenvects()
eigenvalues = [ev[0] for ev in eigen_data]
eigenvectors = [ev[2][0] for ev in eigen_data]

# Function to plot the ellipsoid based on user input
def plot_ellipsoid(no_val, ne_val, x_val):
    if x_val == 0:
        x_val=1e-9 # avoids errors    
    # Substitute numerical values into eigenvalues and eigenvectors
    substitutions = {no: no_val, ne: ne_val, x: x_val}
    evals_numeric = [e.subs(substitutions).evalf() for e in eigenvalues]
    evecs_numeric = [vec.subs(substitutions).evalf() for vec in eigenvectors]

    # Convert to numpy arrays for plotting and normalize the eigenvectors
    evals_np = np.array([float(e) for e in evals_numeric])
    evecs_np = np.array([[float(val) for val in vec] for vec in evecs_numeric])
    evecs_np = np.array([vec / np.linalg.norm(vec) for vec in evecs_np])  # Normalize eigenvectors

    # Generate points on a unit circle to plot the ellipsoid
    theta = np.linspace(0, 2 * np.pi, 100)
    x_circle = np.cos(theta)
    y_circle = np.sin(theta)


    # Scale the unit circle by the refractive index (eigenvalues)
    x_ellipsoid = evals_np[0] * x_circle
    y_ellipsoid = evals_np[1] * y_circle

    # Rotate the ellipsoid using the normalized eigenvectors
    ellipse_points = np.array([x_ellipsoid, y_ellipsoid])
    rotated_points = evecs_np @ ellipse_points

    # Plot the ellipsoid
    fig, ax = plt.subplots(figsize=(6, 6))
    ax.plot(rotated_points[0, :], rotated_points[1, :], color='cyan')
    ax.axhline(0, color='black', linewidth=0.5)
    ax.axvline(0, color='black', linewidth=0.5)

    # Plot eigenvectors
    origin = np.array([0, 0])
    for i in range(2):
        ax.quiver(*origin, *evecs_np[i]*evals_np[i], scale=1, scale_units='xy', color='r', angles='xy', label=f"n{i+1}={evals_np[i]:.5f}")

    ax.set_aspect('equal', 'box')
    ax.set_xlabel("$X$")
    ax.set_ylabel("$Z$")
    ax.legend()
    plt.title("Refractive index ellipsoid")
    plt.grid(True)
    plt.show()

# Interactive sliders
interact(
    plot_ellipsoid,
    no_val=FloatSlider(value=2.8, min=1, max=3, step=0.1, description="no"),
    ne_val=FloatSlider(value=2.7, min=1, max=3, step=0.1, description="ne"),
    x_val=FloatSlider(value=0.001, min=-0.1, max=0.1, step=0.0001, readout_format='.5f', description="rE")
)


interactive(children=(FloatSlider(value=2.8, description='no', max=3.0, min=1.0), FloatSlider(value=2.7, descr…

<function __main__.plot_ellipsoid(no_val, ne_val, x_val)>